In [1]:
import os

In [2]:
%pwd

'f:\\Complete-MLOps-Bootcamp-by-Krish-Naik\\data-science-project\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Complete-MLOps-Bootcamp-by-Krish-Naik\\data-science-project'

In [72]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path 
    train_data_path: Path 
    test_data_path: Path
    model_name:str
    alpha:float
    l1_ratio:float
    target_column: str 

In [73]:
from src.data_science_project.constants import *
from src.data_science_project.utils.common import read_yaml,create_directories

In [74]:
class ConfigurationManager:
    """
    Configuration manager to handle configurations.
    """
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH, schema_file_path = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """ 
        Returns: ModelTrainerConfig: Configuration for model training.
        """
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [75]:
import pandas as pd
import os
from src.data_science_project import logger
from sklearn.linear_model import ElasticNet
import joblib

In [76]:
class ModelTrainer:
    """
    Model Trainer to train the model.
    """
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        """
        Train the model using the training data.
        """
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        
        

In [77]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    logger.exception(e)
    raise e

[2025-06-23 21:42:16,907: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-23 21:42:16,908: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-23 21:42:16,910: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-23 21:42:16,910: INFO: common: created directory at: artifacts]
[2025-06-23 21:42:16,911: INFO: common: created directory at: artifacts/model_trainer]
